In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from airflow_dag import process_data_with_spark_store_in_mongo

/opt/anaconda3/envs/DistDataSys/lib/python3.10/site-packages/airflow/models/base.py:49 MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)

In [3]:
process_data_with_spark_store_in_mongo()

23/02/27 13:22:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


[2023-02-27 13:22:33,530] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_34.114_-118.407_2022_11.csv'
23/02/27 13:22:39 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


[2023-02-27 13:22:43,540] {mongo_db.py:44} WARNING - INSERT KEY ERROR: 34.114_-118.407_2022-11-01_00:00:00
[2023-02-27 13:22:43,899] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_34.114_-118.407_2022_12.csv'
[2023-02-27 13:22:46,475] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_37.756_-122.443_2022_01.csv'
[2023-02-27 13:22:49,049] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_37.756_-122.443_2022_02.csv'
[2023-02-27 13:22:50,997] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_37.756_-122.443_2022_03.csv'
[2023-02-27 13:22:53,106] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_37.756_-122.443_2022_04.csv'
[2023-02-27 13:22:55,042] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_37.756_-122.443_2022_05.csv'
[2023-02-27 13:22:57,189] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_37.756_-122.443_2022_06.csv'
[2023-02-27 13:22:59,366] {airflow_dag.py:93} INFO - Processing: gc

[2023-02-27 13:25:17,822] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_37.972_-122.002_2022_01.csv'


[2023-02-27 13:25:22,799] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_37.972_-122.002_2022_02.csv'
[2023-02-27 13:25:28,383] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_37.972_-122.002_2022_03.csv'


[2023-02-27 13:25:43,102] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_37.972_-122.002_2022_04.csv'
[2023-02-27 13:25:45,929] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_37.972_-122.002_2022_05.csv'
[2023-02-27 13:25:47,838] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_37.972_-122.002_2022_06.csv'
[2023-02-27 13:25:50,473] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_37.972_-122.002_2022_07.csv'
[2023-02-27 13:25:52,716] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_37.972_-122.002_2022_08.csv'
[2023-02-27 13:25:54,607] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_37.972_-122.002_2022_09.csv'
[2023-02-27 13:25:56,506] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_37.972_-122.002_2022_10.csv'
[2023-02-27 13:25:58,252] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_37.972_-122.002_2022_11.csv'
[2023-02-27 13:25:59,737] {mongo_db.py:44} WARNING - INS

[2023-02-27 13:36:13,390] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_33.924_-117.204_2022_09.csv'
[2023-02-27 13:36:17,333] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_33.924_-117.204_2022_10.csv'
[2023-02-27 13:36:20,424] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_33.924_-117.204_2022_11.csv'
[2023-02-27 13:36:22,700] {mongo_db.py:44} WARNING - INSERT KEY ERROR: 33.924_-117.204_2022-11-06_01:00:00
[2023-02-27 13:36:23,036] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_33.924_-117.204_2022_12.csv'
[2023-02-27 13:36:26,403] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_34.428_-119.72_2022_01.csv'
[2023-02-27 13:36:29,891] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_34.428_-119.72_2022_02.csv'
[2023-02-27 13:36:33,737] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_34.428_-119.72_2022_03.csv'
[2023-02-27 13:36:36,307] {airflow_dag.py:93} INFO - Processing: gcp_f